# Convert CSV to VTP

In [24]:
import pandas as pd
import numpy as np

import pathlib
from io import BytesIO

import numpy as np

def read_stl(filename: pathlib.Path) -> tuple[np.ndarray, np.ndarray]:
    """Read buffer content as STL file

    Args:
        buff (io.BufferedReader): buffer to read from

    Returns:
        tuple[np.ndarray, np.ndarray]: return STL representation as (triangles, normals).
    """

    # pass header
    with open(filename, "rb") as f:
        buff = BytesIO(f.read())

    buff.read(80)  # Header
    # Read number of triangles

    n_triangles = np.frombuffer(buff.read(4), dtype=np.uint32)[0]
    if n_triangles == 0:
        raise ValueError("Unable to read number of triangles as 0")

    triangles = np.empty((n_triangles, 3, 3), dtype=np.float32)
    normals = np.empty((n_triangles, 3), dtype=np.float32)

    for idx in range(n_triangles):
        content = buff.read(50)
        normal = np.frombuffer(content[0:12], dtype=np.float32)
        triangle = np.frombuffer(content[12:48], dtype=np.float32).reshape((3, 3))
        triangles[idx] = triangle
        normals[idx] = normal

    return triangles, normals

In [25]:
filename_cp = "CAARC_example_CAARC_015/000/probes/hist_series/body_Pressure_body/bodies.body_Pressure_body.cp.csv"
filename_stl = "CAARC_example_CAARC_015/000/probes/hist_series/body_Pressure_body/bodies.body_Pressure_body.geometry.stl"
filename_vtp_out = "/home/ubuntu/Documents/Codigos/AeroSim/cfdmod/CAARC_example_CAARC_015/000/probes/hist_series/body_Pressure_body/body_cp.vtp"

In [26]:
triangles, normals = read_stl(pathlib.Path(filename_stl))
len(triangles), len(normals)

(2000, 2000)

In [27]:
df_cp = pd.read_csv(filename_cp)
df_cp

,time_step,0,1,2,3,4,5,6,7,8,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,atm
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.032995,1.680243,-0.783543,0.101705,0.835376,1.530131,0.937807,-1.595148,-0.810110,-1.592589,...,-0.909072,-0.046559,-0.213638,0.091350,0.031148,0.022410,-0.221080,-0.130006,-0.048883,0.000000
2,0.065990,1.606081,-0.732091,0.152090,0.630951,1.007039,0.836698,-1.561727,-0.673584,-1.154345,...,-0.768378,-0.075851,-0.072795,0.092391,-0.088031,-0.101754,-0.226453,-0.016264,-0.006963,0.000000
3,0.098985,1.207161,-0.795452,0.119005,0.525440,0.923792,0.789648,-0.964801,-0.710103,-0.979709,...,-0.741483,-0.112781,-0.078441,0.268901,0.079902,-0.229933,-0.176778,-0.121473,-0.141983,0.000000
4,0.131980,0.942938,-1.289702,0.213471,0.711727,1.137594,0.997186,-0.905042,-0.949297,-1.193237,...,-1.099813,0.158364,0.321371,0.094755,0.047920,-0.221080,-0.343196,0.142998,0.194336,-0.007812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,32.862965,0.214413,-0.218190,-0.198802,0.061290,0.242045,0.101541,-0.147790,-0.125575,-0.172608,...,-0.179816,-0.122394,-0.119822,-0.196746,-0.197334,-0.194073,-0.193615,-0.123584,-0.126262,6018.289000
997,32.895958,0.213609,-0.219130,-0.199443,0.061305,0.242339,0.099961,-0.146983,-0.124846,-0.171785,...,-0.178928,-0.123743,-0.120653,-0.196957,-0.197374,-0.194539,-0.193422,-0.124547,-0.127093,6018.922000
998,32.928955,0.214564,-0.220163,-0.199375,0.059832,0.242061,0.100239,-0.145195,-0.125185,-0.173045,...,-0.180599,-0.124128,-0.121705,-0.197087,-0.197293,-0.194375,-0.193315,-0.124425,-0.126640,6018.304700
999,32.961950,0.213653,-0.219021,-0.198292,0.060673,0.240400,0.101567,-0.145255,-0.123849,-0.173662,...,-0.180573,-0.123730,-0.120713,-0.196075,-0.196694,-0.194890,-0.193193,-0.124164,-0.126598,6016.758000


In [28]:
columns_probes = [c for c in df_cp.columns if c.isnumeric()]
assert len(columns_probes) == len(triangles)

len(columns_probes)

2000

In [29]:
scalar_name = "cp_mean"
point_idx = [int(c) for c in columns_probes]
cp_export = df_cp[columns_probes].mean().to_numpy()
cp_export.shape

(2000,)

In [30]:
from vtk import (
    vtkAppendPolyData,
    vtkCellArray,
    vtkFloatArray,
    vtkIdList,
    vtkPoints,
    vtkPolyData,
    vtkXMLPolyDataWriter,
)

def _mkVtkIdList(it) -> vtkIdList:
    """Makes a vtkIdList from a Python iterable. I'm kinda surprised that
     this is necessary, since I assumed that this kind of thing would
     have been built into the wrapper and happen transparently, but it
     seems not.

    Args:
        it (Iterable): A python iterable.

    Returns:
        vtkIdList: A vtkIdList
    """
    vil = vtkIdList()
    for i in it:
        vil.InsertNextId(int(i))
    return vil

def triangles2vertices_and_indexes(triangles: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    vertices = []
    triangles_indexes = np.arange(triangles.shape[0] * 3).reshape(triangles.shape[:2])
    for t in triangles:
        for v in t:
            vertices.append(v)
    vertices = np.array(vertices)
    return vertices, triangles_indexes


def create_polydata_for_cell_data(point_idx: np.ndarray, data: np.ndarray, triangles: np.ndarray, scalar_name: str) -> vtkPolyData:
    """Creates a vtkPolyData for cell data combined with mesh description

    Args:
        point_idx (np.ndarray): Index of points for data, as triangles indexes
        data (np.ndarray): Compiled cell data
        triangles (np.ndarray): Triangles for mesh

    Returns:
        vtkPolyData: Extracted polydata
    """
    # We'll create the building blocks of polydata including data attributes.
    polyData = vtkPolyData()
    points = vtkPoints()
    polys = vtkCellArray()

    # Load the point, cell, and data attributes.
    vertices, triangles_indexes = triangles2vertices_and_indexes(triangles)
    for i, xi in enumerate(vertices):
        points.InsertPoint(i, xi)
    for pt in triangles_indexes:
        polys.InsertNextCell(_mkVtkIdList(pt))

    # We now assign the pieces to the vtkPolyData.
    polyData.SetPoints(points)
    polyData.SetPolys(polys)

    scalars = vtkFloatArray()
    scalars.SetName(scalar_name)

    for (scalar_index, scalar_data) in zip(point_idx, data):
        # Matrix form dataframe, scalar is in rows
        scalars.InsertTuple1(scalar_index, scalar_data)
    polyData.GetCellData().AddArray(scalars)

    return polyData

def write_polydata(output_filename: pathlib.Path, poly_data: vtkPolyData | vtkAppendPolyData):
    """Writes a polydata object to file output

    Args:
        output_filename (pathlib.Path): Output file path
        poly_data (vtkPolyData | vtkAppendPolyData): Polydata object
    """
    writer = vtkXMLPolyDataWriter()
    output_filename.parent.mkdir(parents=True, exist_ok=True)
    writer.SetFileName(output_filename.as_posix())
    if isinstance(poly_data, vtkPolyData):
        writer.SetInputData(poly_data)
    else:
        writer.SetInputData(poly_data.GetOutput())
    writer.SetDataModeToAscii()
    writer.Write()


In [31]:
polydata = create_polydata_for_cell_data(point_idx, cp_export, triangles, scalar_name)

write_polydata(pathlib.Path(filename_vtp_out), polydata)